In [1]:
import cv2
import os

# Set the name for the dataset directory
dataset_name = "my_face"
dataset_path = f"dataset/{dataset_name}"

if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

# Load Haar cascade for face detection
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def capture_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return None
    # Crop just the face region
    for (x, y, w, h) in faces:
        cropped_face = img[y:y+h, x:x+w]
        return cropped_face

cap = cv2.VideoCapture(0)
count = 0
total_images = 100

print("Please look at the camera...")

while True:
    ret, frame = cap.read()
    if capture_face(frame) is not None:
        count += 1
        face = cv2.resize(capture_face(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        file_name_path = f"{dataset_path}/{count}.jpg"
        cv2.imwrite(file_name_path, face)

        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2)
        cv2.imshow("Face Dataset", face)
    else:
        print("Face not found")

    if cv2.waitKey(1) == 13 or count == total_images: # Press Enter to exit early
        break

cap.release()
cv2.destroyAllWindows()
print(f"Collected {count} face samples. Dataset stored at {dataset_path}")


<>:17: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?


ModuleNotFoundError: No module named 'cv2'

In [2]:
import cv2
import numpy as np
import os

# Path to face dataset
dataset_path = "dataset/my_face"

# Check if dataset exists
if not os.path.exists(dataset_path):
    print(f"Error: Dataset path '{dataset_path}' does not exist!")
    exit()

# Prepare data for training
faces = []
labels = []

# Label for your face
label = 1

print("Loading face images...")
image_count = 0

for filename in os.listdir(dataset_path):
    if filename.endswith(".jpg"):
        img_path = os.path.join(dataset_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        if img is not None:
            faces.append(img)
            labels.append(label)
            image_count += 1
        else:
            print(f"Warning: Could not read {filename}")

if len(faces) == 0:
    print("Error: No face images found in dataset!")
    exit()

print(f"Loaded {image_count} face images.")

faces_np = np.array(faces)
labels_np = np.array(labels)

# Create and train the LBPH face recognizer
print("Training model...")
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(faces_np, labels_np)
face_recognizer.save("trained_model.yml")

print("✓ Training complete. Model saved as trained_model.yml.")

Loading face images...
Loaded 100 face images.
Training model...


AttributeError: module 'cv2' has no attribute 'face'

In [1]:
import cv2
import numpy as np

# Load the trained model
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.read("trained_model.yml")

# Load face cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Start webcam
cap = cv2.VideoCapture(0)

print("Looking for your face...")

# Recognition parameters
confidence_threshold = 70  # Lower = stricter, Higher = more lenient
recognition_count = 0
required_recognitions = 5  # Need 5 consecutive recognitions to unlock

unlocked = False

while True:
    ret, frame = cap.read()
    
    # Check if frame was read successfully
    if not ret or frame is None:
        print("Error: Could not read frame from camera")
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        # Extract face region
        face_roi = gray[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (200, 200))
        
        # Predict
        label, confidence = face_recognizer.predict(face_roi)
        
        # Draw rectangle
        if confidence < confidence_threshold and label == 1:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, f"Recognized! ({int(confidence)})", (x, y-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            recognition_count += 1
            
            if recognition_count >= required_recognitions:
                print("✓ Face verified! Access granted.")
                unlocked = True
                break  # Break from for loop
        else:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            cv2.putText(frame, f"Unknown ({int(confidence)})", (x, y-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            recognition_count = 0  # Reset counter
    
    # Break from while loop if unlocked
    if unlocked:
        break
    
    cv2.imshow("Face Recognition", frame)
    
    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

if unlocked:
    print("🔓 System unlocked successfully!")
    # HERE: Add your unlock command for your OS
else:
    print("Recognition stopped.")

Looking for your face...
✓ Face verified! Access granted.
🔓 System unlocked successfully!


In [5]:
import cv2
import numpy as np
import pyautogui
import time

# Load the trained model
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.read("trained_model.yml")  # path to your model

# Load Haar cascade for face detection
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        return gray[y:y+h, x:x+w]
    return None

cap = cv2.VideoCapture(0)
unlock_threshold = 55  # Adjust based on training and testing

print("Face unlock running. Looking for your face...")

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    face = detect_face(frame)
    if face is not None:
        face = cv2.resize(face, (200, 200))
        label, confidence = face_recognizer.predict(face)
        print(f"Confidence: {confidence}")

        if confidence < unlock_threshold:
            print("Access Granted: Laptop Unlocked!")
            # You can implement your unlock logic here
            # For example, simulate Ctrl+Alt+Del or show a notification
            break
        else:
            print("Access Denied: Locking laptop")
            # Lock screen for Windows (this command)
            pyautogui.hotkey('win', 'l')
            time.sleep(5)
    else:
        print("No face detected...")

    cv2.imshow("Face Recognition", frame)
    if cv2.waitKey(1) == 13:  # Press Enter to exit
        break

cap.release()
cv2.destroyAllWindows()


Face unlock running. Looking for your face...
Confidence: 46.63398281478578
Access Granted: Laptop Unlocked!
